In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [24]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (105.67 seconds): Collecting URLs... yahoo               

100.00% (1.94 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,25986,25922,1265
reuters,1158,1158,161
investing.com,925,925,114
dailymail,2885,2885,97
euronews,955,955,89
...,...,...,...
blastingnews,123,123,0
ccn,0,0,0
afr,451,451,0
realclearpolitics,292,270,0


,url,pubs,actual_pub
0,https://politico.com/news/2019/10/17/pelosi-de...,"politico, realclearpolitics",politico
1,https://politico.com/news/2019/10/16/elizabeth...,"politico, realclearpolitics",politico
2,https://politico.com/news/2019/10/17/trump-syr...,"politico, realclearpolitics",politico
3,https://politico.com/magazine/story/2019/10/15...,"politico, realclearpolitics",politico
4,https://politico.com/news/2019/10/15/trump-con...,"politico, realclearpolitics",politico
...,...,...,...
59,https://msnbc.com/msnbc/watch/mulvaney-says-tr...,"msnbc, nbcnews",msnbc
60,https://msnbc.com/ali-velshi/watch/romney-aban...,"msnbc, nbcnews",msnbc
61,https://nbcnews.com/politics/politics-news/pen...,"msnbc, nbcnews",nbcnews
62,https://nbcnews.com/news/latino/house-holds-it...,"msnbc, nbcnews",nbcnews


# 2. 해당 URL의 뉴스를 다운로드한다

In [ ]:
download_result = crawler.download(); download_result

5.77% (20.01 seconds): Downloading... breitbart           

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


96.36% (331.54 seconds): Downloading... dailymail           